In [1]:
import re

identifiers = '[_a-zA-Z][_a-zA-Z0-9]*'
string = '"[a-zA-Z][a-zA-Z0-9]*"'
ints = '[0-9][0-9]*'
floatsone = '\d+\.\d*'
floatstwo = '\.\d+'
octal = '[0][0-7]{1,2}'

In [2]:
# Available operations in the language
operations = {
'=':'ASSIGN_OPER', "!":"NEGATIVE",
'+':'ADDITION_OPER', '-':'SUB_OPER', 
'*':'MULT_OPER', '/':'DIV_OPER', 
'>':"GREATER_THAN", '<':"LESS_THAN",
'%':'MOD_OPER', '\'':'SINGLE_QUOTES', 
'"':'DOUBLE_QUOTES', '(':'LEFT_PARANTHESIS',
')':'RIGHT_PARANTHESIS', ":":"COLON",
'{':'LEFT_CURL','}':'RIGHT_CURL', 
';':'DELIMETER'
}

In [3]:
# These are the keywords of our language
keywords = {
'void':'VOID_CODE', 'main':'MAIN_CODE', 
'int':"INT_CODE", 'char':'CHAR_CODE',
'unsigned':'UNSIGNED_CODE', 'signed':'SIGNED_CODE',
'short':'SHORT_CODE', 'long':'LONG_CODE',
'float':'FLOAT_CODE', 'double':'DOUBLE_CODE', 
'struct':'STRUCT_CODE', 'default':'DEFAULT_CODE',
'break':"BREAK_CODE", 'continue':'CONTINUE_CODE',
'if':'IF_CODE','else':'ELSE_CODE',
'for':'FOR_CODE','foreach':'FOREACH_CODE',
'switch':'SWITCH_CODE', 'case':'CASE_CODE',
'enum':"ENUM_CODE", 'const':'CONST_CODE', 
'while':'WHILE_CODE', 'do':'DO_CODE',
'goto':'GOTO_CODE','sizeof':'SIZEOF_CODE',
'return':'RETURN_CODE','static':'STATIC_CODE'
}

In [4]:
# Just to store values in the process
value = ''

# This is an array for the parsed values
parses = []

with open('input.txt', 'r') as file:

    while True:
        read_character = file.read(1)

        if read_character == "":
            break

        token = read_character
        identifier = re.fullmatch(identifiers, read_character)

        if identifier:
            val = 'IDENTIFIER'
            
        integers = re.fullmatch(ints, read_character)
        floats = re.fullmatch(floatsone, read_character)
        strings = re.fullmatch(string, read_character)

        if integers:
            val='INTEGER'
            
        if read_character == '"':
            next_character = ''
            while next_character != '"':
                next_character = file.read(1)
                temp = token + next_character
                token = temp
                if(next_character == '"'):
                    string_match = re.fullmatch(string, temp)
                    if string_match:
                        val = "STRING" 
                        
        if val == "STRING":
            if token != " ":
                parses.append((token,val))
                token=''
                val=""

        if read_character == ".":
            while read_character != '':
                next_character = file.read(1)
                if next_character == "":
                    break
                if next_character == " ":
                    break
                temp = token + next_character
                float_dot = re.fullmatch(floatstwo, temp)
                if float_dot:
                    token = temp
                    val = "FLOAT" 
                if next_character in operations.keys():
                    value = operations[next_character]  
                else:
                    break

        if read_character != "" and read_character in operations.keys():
            val = operations[read_character]
            parses.append((read_character, val))  

        while read_character != '' and identifier: 
            next_character = file.read(1)
            if next_character == "":
                break
            if next_character == " ":
                break
            temp = token + next_character
            identifier = re.fullmatch(identifiers, temp)
            if identifier:
                token = temp
                val="IDENTIFIER"
            if next_character in operations.keys():
                value = operations[next_character] 

        while read_character != '' and integers or floats:
            next_character = file.read(1)
            if next_character == "":
                break
            if next_character == " ":
                break
            temp = token + next_character
            integers = re.fullmatch(ints, temp)
            floats = re.fullmatch(floatsone, temp)
            if floats:
                token = temp
                val = "FLOAT"
            elif integers:
                token = temp
                val="INTEGER"
            if next_character in operations.keys():
                value = operations[next_character]
        
        if val == "IDENTIFIER" or val == "INTEGER" or val=="FLOAT" or val=="OCTAL"and token != " " and token != "\n":
            if token != " ":
                parses.append((token,val))
            token=''
            val=""
        if value and next_character != " " and next_character != "\n":
            parses.append((next_character, value))
            value=''
        token=''
        val=""
        if read_character== "":
            break 

In [5]:
for i in range(len(parses)):
    if parses[i][1] == 'IDENTIFIER':
        if parses[i][0] in keywords.keys():
            parse_temp = list([parses[i][0], keywords[parses[i][0]]])
            parses[i] = tuple(parse_temp)
        elif parses[i][0] == 'true':
            parse_temp = list(['true','BOOLEAN'])
            parses[i] = tuple(parse_temp)
        elif parses[i][0] == 'false':
            parse_temp = list(['false', 'BOOLEAN'])
            parses[i] = tuple(parse_temp)

i = 0
while i < len(parses):
    if parses[i][0] == "=" and parses[i+1][0] == "=":
        parses.pop(i+1)
        parses[i] = ("==", 'EQUALITY_OPERATOR')
    else:
        i +=1

i = 0
while i < len(parses):
    if parses[i][0] == ">" and parses[i+1][0] == "=":
        parses.pop(i+1)
        parses[i] = (">=", 'GREAT_EQUAL')
    else:
        i +=1

i = 0
while i < len(parses):
    if parses[i][0] == "<" and parses[i+1][0] == "=":
        parses.pop(i+1)
        parses[i] = ("<=", 'LESS_EQUAL')
    else:
        i +=1

i = 0
while i < len(parses):
    if parses[i][0] == "!" and parses[i+1][0] == "=":
        parses.pop(i+1)
        parses[i] = ("!=", 'NOT_EQUAL')
    else:
        i +=1
for i in parses:
    print(i)

for i in range(len(parses)):
        if parses[i][0] == 'true':
            parse_temp = list(['1','INTEGER'])
            parses[i] = tuple(parse_temp)
        elif parses[i][0] == 'false':
            parse_temp = list(['0', 'INTEGER'])
            parses[i] = tuple(parse_temp)

type_current = 'INTEGER'
for i in range(len(parses)):
    if parses[i][1] == 'FLOAT':
        type_current = 'FLOAT'
final_value = None

('void', 'VOID_CODE')
('main', 'MAIN_CODE')
('(', 'LEFT_PARANTHESIS')
(')', 'RIGHT_PARANTHESIS')
('{', 'LEFT_CURL')
('int', 'INT_CODE')
('_test123', 'IDENTIFIER')
('=', 'ASSIGN_OPER')
('123', 'INTEGER')
(';', 'DELIMETER')
('return', 'RETURN_CODE')
('"hello"', 'STRING')
('"', 'DOUBLE_QUOTES')
(';', 'DELIMETER')
('}', 'RIGHT_CURL')


In [6]:
def lex():
    global nextToken
    global nextValue
    if parses:
        val = parses.pop(0)
        nextToken = val[1]
        nextValue = val[0]
        print(nextToken)   

def expr():
    global final_value
    print("STARTED <expr>")
    if nextToken == 'INTEGER':
        final_value = int(nextValue)
    elif nextToken == 'FLOAT':
        final_value = float(nextValue)

    term()
    while (nextToken == 'ADDITION_OPER' or nextToken == 'sub_op'):
        if nextToken == 'ADDITION_OPER':
            lex()
            term()
        elif nextToken == 'sub_op':
            lex()
            term()
    print("FINISHED <expr>")
    
def term():
    global final_value
    print("STARTED <term>")
    factor()
    while(nextToken == 'MULT_OPER' or nextToken == 'DIV_OPER' or nextToken == 'MOD_OPER'):
        if nextToken == 'MULT_OPER':
            lex()
            if nextToken == 'INTEGER':
                final_value *= int(nextValue)
            elif nextToken == 'FLOAT':
                final_value *= float(nextValue)
            factor()
        elif nextToken == 'DIV_OPER' or nextToken=='MOD_OPER':
            lex()
            if nextToken == 'INTEGER' and int(nextValue) == 0 or nextToken == 'FLOAT' and float(nextValue) == 0.0:
                print('Zero Division Error')
                exit()
            factor()
        
    print("FINISHED <term>")

def factor():
    print("STARTED <factor>")
    if (nextToken == 'IDENTIFIER' or nextToken == 'INTEGER' or nextToken == 'FLOAT'):
        lex()
    elif nextToken == 'LEFT_PARANTHESIS':
        lex()
        expr()
        if nextToken == 'RIGHT_PARANTHESIS':
            lex()
        else:
            error()
    else:
        error()
    print("FINISHED <factor>")

def error():
    print("Something went wrong")
    exit()

def assign():
    print('STARTED <assignment>')
    global correct_type
    lex()
    if nextToken != "ASSIGN_KEY":
        error()
    lex()
    if nextValue == 'int':
        correct_type = 'INTEGER'
        lex()
        if nextToken != 'IDENTIFIER':
            error()
        lex()
        if nextToken != 'ASSIGN_OPER':
            error()
        lex()
        expr()
        if nextToken != 'delimiter':
            error()
        if correct_type == type_current:
            print('Successful assignment')
        else:
            print('TypeError: Cannot be assigned')
            exit()    
        print('FINISHED <assingment>')
    elif nextValue == 'float':
        correct_type = 'FLOAT'
        lex()
        if nextToken != 'IDENTIFIER':
            error()
        lex()
        if nextToken != 'ASSIGN_OPER':
            error()
        lex()
        expr()
        if nextToken != 'delimiter':
            error()
        if correct_type == type_current:
            print('Successful assignment')
        else:
            print('TypeError: Cannot be assigned')
            exit()    
        print('FINISHED <assignment>')

In [7]:
assign()  

STARTED <assignment>
VOID_CODE
Something went wrong
MAIN_CODE
